In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

import pandas as pd

data = pd.read_csv("star_classification.csv", sep=',')
output = data['class']
input = data.drop(['class','rerun_ID'], axis=1)

input_max = input.max()
input_min = input.min()
input_normalized = (input - input_min) / (input_max - input_min)
df = pd.DataFrame(input_normalized)

# # Compute the correlation matrix
correlation_matrix = df.corr()

# # Visualize the correlation matrix using a heatmap
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.show()
dataset = pd.concat([input_normalized, output], axis=1)

# Shuffle the dataset (by shuffling rows)
shuffled_dataset = dataset.sample(frac=1, random_state=42).reset_index(drop=True)

# Calculate the index to split the dataset (90% for training and 10% for testing)
train_size = int(0.9 * len(shuffled_dataset))

# Split the dataset into training and testing sets
train_data = shuffled_dataset[:train_size]
test_data = shuffled_dataset[train_size:]

# Separate features and labels for training and testing
input_normalized = train_data.drop('class', axis=1)
output = train_data['class']

X_test = test_data.drop('class', axis=1)
Y_test = test_data['class']

import matplotlib.pyplot as plt

# Plot histogram of the classes
output.value_counts().plot(kind='bar', color='skyblue')
plt.title('Class Distribution Before Oversampling Minority Cases')
plt.xlabel('Classes')
plt.ylabel('Frequency')
plt.show()

# Check class distribution
class_counts = output.value_counts()
print(class_counts)

max_class_size = class_counts.max()
classes = class_counts.index

print(f"Majority Class Size: {max_class_size}")

# oversample
oversampled_data = []
# Loop through each class
for cls in classes:
    class_data = input_normalized[output == cls]  # Get data for the current class
    oversampled_class = class_data.sample(max_class_size, replace=True, random_state=42)  # Oversample with replacement
    oversampled_data.append(oversampled_class)

# Combine all oversampled classes
oversampled_input = pd.concat(oversampled_data, axis=0)

# Create the corresponding labels
oversampled_output = pd.Series(
    [cls for cls in classes for _ in range(max_class_size)],
    name='class'
)


#shuffle
# Reset indices of oversampled data and labels
oversampled_input = oversampled_input.reset_index(drop=True)
oversampled_output = oversampled_output.reset_index(drop=True)

# Combine input and output
balanced_data = pd.concat([oversampled_input, oversampled_output], axis=1)

# Shuffle the dataset
balanced_data = balanced_data.sample(frac=1, random_state=42).reset_index(drop=True)
input = data.drop(['class'], axis=1)

data = balanced_data

output = data['class']

input = data.drop(['class'], axis=1)


input_max = input.max()

input_min = input.min()

input_normalized = (input - input_min) / (input_max - input_min)

import matplotlib.pyplot as plt


balanced_data['class'].value_counts().plot(kind='bar', color='lightblue', edgecolor='black')
plt.title('Class Distribution After Oversampling')
plt.xlabel('Classes')
plt.ylabel('Frequency')
plt.xticks(rotation=0)
plt.show()

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

def one_hot_representation(class_array):
    data = sorted(list(set(class_array)))
    num_classes = len(data)
    one_hot = np.eye(num_classes)
    # Create a mapping from label to index
    label_to_index = {label: idx for idx, label in enumerate(data)}

    # Generate one-hot encoded matrix
    one_hot_encoded = one_hot[[label_to_index[label] for label in class_array]]


    # one_hot_encoded = np.array([one_hot[:,data.index(label)] for label in class_array])
    # one=np.where(class_array == data[0], one_hot[:,2] , 0)  
    label = np.split(one_hot,one_hot.shape[1],axis=0)
    
    return one_hot_encoded , label
def sigmoid(x):
    return 1 / (1 + np.exp(-x))  
    
    
    return one_hot_encoded , label
def activation_derivative(z, activation):
    if activation == "relu":
       
        derivative = (z > 0).astype(float)
    elif activation == "sigmoid":
        derivative = sigmoid(z)*(1-sigmoid(z))
    else:
        derivative = 0
    return np.diag(derivative)
def loss(y_true, output , h_layers):
    
    epsilon = 1e-9  # Small value to prevent log(0)
    # Clip y_pred to avoid log(0)
    y_pred = np.clip(output["A" + str(h_layers+1)], epsilon, 1. - epsilon)
    
    # Compute cross-entropy loss
    
    # print(y_pred)
    loss = -np.sum(np.dot(y_true.T , np.log(y_pred))) / y_true.shape[0]
    return loss
def compute_confusion_matrix(y_test , y_pred):
    
    Y_pred_classes = np.argmax(y_pred, axis=1)
    Y_test_classes = np.argmax(y_test, axis=1)
 
    # Create a confusion matrix
    conf_matrix = confusion_matrix(Y_test_classes, Y_pred_classes)

# Display the confusion matrix
    disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=["Galaxy", "Star", "QSO"])
    disp.plot(cmap="Blues")
    plt.title("Confusion Matrix for NN")
    plt.show()
    
    return conf_matrix
def softmax(z):
    # Subtracting the max of z for numerical stability
    # This prevents overflow by scaling the logits
    
    z_exp = np.exp(z - np.max(z, axis=1, keepdims=True))  # Exponentiate the logits (with stability adjustment)
    # print(z_exp)
    # print(np.sum(z_exp, axis=1, keepdims=True))
    return z_exp / np.sum(z_exp, axis=1, keepdims=True)  # Normalize the values to sum to 1     

def accuracy_score(y_test, y_pred):
    # Check if the lengths of y_test and y_pred are the same
    if len(y_test) != len(y_pred):
        raise ValueError("The length of y_test and y_pred must be the same.")
    y_test_class = np.argmax(y_test, axis=1)
    y_pred_class = np.argmax(y_pred, axis=1)
    # Count the number of correct predictions (where y_test[i] == y_pred[i])
    correct_predictions = np.sum(y_test_class == y_pred_class)
    
    # Calculate accuracy
    accuracy = correct_predictions / len(y_test)
    
    return accuracy





def f1(y_test, y_pred, class_labels):
    # Initialize F1 scores list for each class
    f1_scores = []
    
    # For each class label
    for label in class_labels:
        TP = np.sum((y_test == label) & (y_pred == label))
        FP = np.sum((y_test != label) & (y_pred == label))
        FN = np.sum((y_test == label) & (y_pred != label))

        precision = TP / (TP + FP) if (TP + FP) > 0 else 0
        recall = TP / (TP + FN) if (TP + FN) > 0 else 0
        
        f1_class = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        f1_scores.append(f1_class)
    
    # Return the average F1 score (macro average in this case)
    return np.mean(f1_scores)

class NN:
    def __init__(self, learning_rate=0.001, epoch=1000, num_classes= 3,num_hidden=[10], activation = "relu"):
        self.lr = learning_rate
        self.epoch = epoch
        self.classes = num_classes
        self.parameters = {}
        self.losses = []
        self.h_layers = len(num_hidden) # Number of hidden layers
        self.hidden_n = num_hidden # List of hidden layer neurons
        self.activation = activation # Chosen activation function
        
    

    def fit(self, X, y, batch_size=32, SGD=False):
        n_samples, n_features = X.shape
        
        
        self.parameters["W1"] = np.random.randn(n_features, self.hidden_n[0]) * 0.01  # Weight matrix
        self.parameters["b1"] = np.zeros((1, self.hidden_n[0]))  # Bias vector
        if self.h_layers > 1:
            for l in range(1, self.h_layers):
                
                self.parameters["W" + str(l+1)] = np.random.randn(self.hidden_n[l-1], self.hidden_n[l]) * 0.01  # Weight matrix
                self.parameters["b" + str(l+1)] = np.zeros((1, self.hidden_n[l]))  # Bias vector
        self.parameters["W"+ str(self.h_layers+1)] = np.random.randn(self.hidden_n[-1], self.classes) * 0.01  # Weight matrix
        self.parameters["b"+ str(self.h_layers+1)] = np.zeros((1, self.classes))  # Bias vector
        
        
        for epoch in range(self.epoch):
            if SGD:
                indices = np.random.permutation(n_samples)
                X_shuffled = X[indices]
                y_shuffled = y[indices]
                n_batch= X_shuffled.shape[0] // batch_size 
                for i in range(n_batch):
                    xi = X_shuffled[(i*batch_size):((i+1)*batch_size)]  # Get one batch
                    yi = y_shuffled[(i*batch_size):((i+1)*batch_size)]  # Get the corresponding label
                    delta = {}
                        
                    output, ind_field = self.feed_forward(xi,self.activation)
                    self.losses.append(loss(yi, output, self.h_layers))
                        
                    delta["d" + str(self.h_layers + 1)] = output["A" + str(self.h_layers + 1)] - yi
                   
                    for i in range(self.h_layers, 0, -1):
                        delta["d" + str(i)] = np.dot(delta["d" + str(i + 1)],  self.parameters["W"+ str(i+1)].T) * activation_derivative(ind_field["z" + str(i)], self.activation) 
                    self.update_parameters(delta,self.h_layers, output)
                if X.shape[0] % batch_size != 0:      
                    xi = X_shuffled[(n_batch*batch_size)::]  
                    yi = y_shuffled[(n_batch*batch_size)::]  
                    delta = {}
                        
                    output, ind_field = self.feed_forward(xi,self.activation)
                    self.losses.append(loss(yi, output,self.h_layers))
                        
                    delta["d" + str(self.h_layers + 1)] = output["A" + str(self.h_layers + 1)]- yi
                    for i in range(self.h_layers, 0, -1):
                        delta["d" + str(i)] = np.dot(delta["d" + str(i + 1)],  self.parameters["W"+ str(i+1)].T) * activation_derivative(ind_field["z" + str(i)], self.activation) 
                    self.update_parameters(delta,self.h_layers, output)
            else:
                X_shuffled = X
                y_shuffled = y
                n_batch= X_shuffled.shape[0] // batch_size 
                for i in range(n_batch):
                    xi = X_shuffled[(i*batch_size):((i+1)*batch_size)]
                    yi = y_shuffled[(i*batch_size):((i+1)*batch_size)] 
                    delta = {}
                        
                    output, ind_field = self.feed_forward(xi,self.activation)
                    self.losses.append(loss(yi, output, self.h_layers))
                    
                    delta["d" + str(self.h_layers + 1)] = output["A" + str(self.h_layers + 1)] - yi
                    
                    for i in range(self.h_layers, 0, -1):
                        # print(self.parameters["W"+ str(i+1)].shape)
                        # print(delta["d" + str(i + 1)].shape)
                        #delta["d" + str(i)] = delta["d" + str(i + 1)] * activation_derivative(ind_field["z"+ str(i)], self.activation) * self.parameters["W"+ str(i+1)]
                        delta["d" + str(i)] = np.dot(delta["d" + str(i + 1)],  self.parameters["W"+ str(i+1)].T) * activation_derivative(ind_field["z" + str(i)], self.activation) 
                        
                    self.update_parameters(delta,self.h_layers, output)
                if X.shape[0] % batch_size != 0:    
                    xi = X_shuffled[(n_batch*batch_size)::]  # Get remaining 
                    yi = y_shuffled[(n_batch*batch_size)::]  # Get the corresponding label
                    delta = {}
                        
                    output, ind_field = self.feed_forward(xi,self.activation)
                    self.losses.append(loss(yi, output, self.h_layers))
                        
                    delta["d" + str(self.h_layers + 1)] = output["A" + str(self.h_layers + 1)] - yi
                    for i in range(self.h_layers, 0, -1):
                        #delta["d" + str(i)] = delta["d" + str(i + 1)] * activation_derivative(ind_field["z" + str(i)], self.activation) * self.parameters["W"+ str(i+1)]
                        delta["d" + str(i)] = np.dot(delta["d" + str(i + 1)],  self.parameters["W"+ str(i+1)].T) * activation_derivative(ind_field["z" + str(i)], self.activation) 

                    self.update_parameters(delta,self.h_layers)
    
    def forward_pass(self,X,i,activation):
        
        if i  == self.h_layers:
  
            
            z = np.dot(X, self.parameters["W" + str(i+1)]) + self.parameters["b" + str(i+1)]
            A = softmax(z)
            
        else:
            
            
            z = np.dot(X, self.parameters["W" + str(i+1)]) + self.parameters["b" + str(i+1)]
            if activation == "relu":
                A = np.maximum(0,z)
            elif activation == "sigmoid":
                A = sigmoid(z)
        return A , z         
            
            
    def feed_forward(self, X , activation):
        output = {}
        ind_field = {}
        output["A" + str(0)] = X
        
        for i in range(self.h_layers+1):
            output["A" + str(i+1)], ind_field["z" + str(i+1)] = self.forward_pass(output["A" + str(i)] ,i, activation)
            
        return output , ind_field
       
    def update_parameters(self, delta , i, output):
        for l in range(i + 1, 0, -1):
            self.parameters['W' + str(l)] = self.parameters['W' + str(l)]  - self.lr * np.dot(output["A" + str(l-1)].T , delta["d" + str(l)] ) 
            self.parameters['b' + str(l)]  = self.parameters['b' + str(l)]  - self.lr * delta["d" + str(l)]
            
    def predict(self, X):
        
        y_prediction = np.zeros((X.shape[0],self.classes))
        output , _ = self.feed_forward(X ,self.activation)
        
        max_i = np.argmax(output['A'+ str(self.h_layers + 1)], axis=1)
        
        
        y_prediction[np.arange(len(output['A'+ str(self.h_layers + 1)])),max_i] = 1
        
        return np.array(y_prediction)
    
def k_fold(X, Y, k=5, shuffle=False, learning=0.001, epoch=(25,), batch_size=80000, sgd=True, hidden_layers=([5], [10], [10, 5]), activations=("relu", "sigmoid")):

    
    # Step 1: Shuffle the data
    if shuffle:
        indices = np.random.permutation(len(X))
       
        X_shuffled = X.iloc[indices]
        Y_shuffled = Y.iloc[indices]
        # num_class = len(set(Y))
        
    else:
        X_shuffled = X
        Y_shuffled = Y
    Y_shuffled_ohr, oh_labels = one_hot_representation(Y_shuffled) # One hot representation of classes
    
    # Step 2: Split data into k folds
    fold_size = len(X) // k

    cm = []
    model_accuracies = []
    model_f1_scores = []
    models = []
    for m in range(len(epoch)):
        epochs  = epoch[m]
        for o in range(len(hidden_layers)):
            hidden_layer = hidden_layers[o]
            for p in range(len(activations)):
                activation = activations[p]
                accuracies = []  # List to store accuracy for each fold
                f1_scores = []
                for i in range(k):
                # Step 3: Define the test set for this fold
                    test_start = i * fold_size
                    test_end = (i + 1) * fold_size if i != k - 1 else len(X)
                    
                # Split data into training and test sets
                    X_train = np.concatenate([X_shuffled[:test_start], X_shuffled[test_end:]], axis=0)
                    Y_train = np.concatenate([Y_shuffled_ohr[:test_start], Y_shuffled_ohr[test_end:]], axis=0)
                    X_test = X_shuffled[test_start:test_end]
                    Y_test = Y_shuffled_ohr[test_start:test_end]
                    
                    
                    # Step 4: Train the model on the training set
                    model = NN(learning_rate=learning, epoch=epochs, num_classes= 3,num_hidden=hidden_layer, activation = activation)
                    
                    batch_s = min(batch_size, X_train.shape[0])
                    
                    model.fit(X_train, Y_train, batch_s, SGD= sgd)
                    
                    # Step 5: Make predictions on the test set
                    Y_pred = model.predict(X_test)
                    
                    # Step 6: Calculate accuracy for this fold
                    accuracy = accuracy_score(Y_test, Y_pred)
                    accuracies.append(accuracy)
      
                    f1_score = f1(Y_test, Y_pred, oh_labels)
                    f1_scores.append(f1_score)
                    #cm.append(compute_confusion_matrix(Y_test, Y_pred))
    # Step 7: Calculate the average accuracy across all folds
                average_accuracy = np.mean(accuracies)
                average_f1_score = np.mean(f1_scores)
                # print(model.losses)
                print("Neural Networks: ")
                print("For learning rate " + str(learning) + " and epoch " + str(epochs)  + " and hidden layers " + str(hidden_layer) + " and activation " + str(activation))
                print("Validation Accuracy: " + str(average_accuracy))
                print("Validation F1 Score: " + str(average_f1_score) + "\n")

                model_accuracies.append(average_accuracy)
                model_f1_scores.append(average_f1_score)
                models.append(model)
            #model_cm.append(cm)
    return model_accuracies, model_f1_scores, models





In [ ]:
Y_test_ohr, oh_lab = one_hot_representation(Y_test)

Acc_nn ,F1_nn ,models_nn=k_fold(input_normalized, output, k=5, shuffle=False, learning=0.001, epoch=(25,), batch_size=1, sgd= False, hidden_layers=([5],), activations = ("relu",))
print("Validation Accuracies are " + str(Acc_nn))
print("Validation F1 Scores are " + str(F1_nn))

best = np.argmax(F1_nn)
best_model = models_nn[best]

predictions = best_model.predict(X_test)
accuracy = accuracy_score(Y_test_ohr, predictions)
f1_score = f1(Y_test_ohr, predictions, oh_lab)
cm = (compute_confusion_matrix(Y_test_ohr, predictions))
print("Test Accuracy: " + str(accuracy))
print("Test F1 Score: " + str(f1_score))